In [1]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
from nltk.stem import SnowballStemmer
from nltk.tokenize import WordPunctTokenizer
import re
from bs4 import BeautifulSoup
part1 = r'@[A-Za-z0-9]+'
part2 = r'https?://[A-Za-z0-9./]+'
combined_part = r'|'.join((part1, part2))

def text_cleaner(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'https\S+', '', text)
    
    soup = BeautifulSoup(text, 'lxml')
    soup_text = soup.get_text()
    stripped = re.sub(combined_part, '', soup_text)
    
    try:
        clean = stripped.replace(u"ï¿½", "?")
    except:
        clean = stripped
    
    clean = re.sub("'", "", clean)
    # remove all non letter and irrelevant punctuation
    clean = re.sub( r'([?!])', r' \1 ', clean)
    clean = re.sub("[^a-zA-Z?!]", " ", clean)
    
    # all letters to lower case
    clean = clean.lower()
    
    #remove all extra spaces
    tok = WordPunctTokenizer()
    words = tok.tokenize(clean)
    
    #join list into a sentence
    return (" ".join(words)).strip()

In [4]:
df=pd.read_csv("hijaako.csv")

In [5]:
print(df['tweet'][0])
text_cleaner(df['tweet'][0])

Philippines Hashtag Battle: #HijaAko Vs #AmaAko - The ASEAN Post https://theaseanpost.com/article/philippines-hashtag-battle-hijaako-vs-amaako?utm_source=dlvr.it&utm_medium=twitter …


'philippines hashtag battle hijaako vs amaako the asean post'

In [6]:
df["tweet"]=df["tweet"].apply(lambda x: text_cleaner(x))

In [7]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out


In [9]:
nlp = spacy.load('en', disable=['parser', 'ner'])

In [11]:
data=lemmatization(df.tweet.values.tolist(), allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [15]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df.tweet.values.tolist()))

In [16]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])



In [60]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=1,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             max_features=5000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(df["tweet"].values.tolist())

In [62]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  0.23795474200057654 %


In [63]:
# Build LDA Model
lda_model = LatentDirichletAllocation( 
                                     n_components=5,# Number of topics
                                      max_iter=15,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=15,
                          mean_change_tol=0.001, n_components=5, n_jobs=-1,
                          perp_tol=0.1, random_state=100, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


In [64]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
print(lda_model.get_params())

Log Likelihood:  -1214423.596052154
Perplexity:  879.2206036724059
{'batch_size': 128, 'doc_topic_prior': None, 'evaluate_every': -1, 'learning_decay': 0.7, 'learning_method': 'online', 'learning_offset': 10.0, 'max_doc_update_iter': 100, 'max_iter': 15, 'mean_change_tol': 0.001, 'n_components': 5, 'n_jobs': -1, 'perp_tol': 0.1, 'random_state': 100, 'topic_word_prior': None, 'total_samples': 1000000.0, 'verbose': 0}


In [67]:
# Define Search Param
search_params = {'n_components': [3,4,5], 'learning_decay': [.7]}

# Init the Model
lda = LatentDirichletAllocation( n_jobs = -1,batch_size=128, evaluate_every = -1,max_iter=10,learning_method='online', random_state=100)

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

GridSearchCV(cv=None, error_score=nan,
             estimator=LatentDirichletAllocation(batch_size=128,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method='online',
                                                 learning_offset=10.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=10,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=-1,
                                                 perp_tol=0.1, random_state=100,
                                                 topic_word_prior=None,
                                                 total_samples=1000000.0,
                              

In [68]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.7, 'n_components': 3}
Best Log Likelihood Score:  -258165.4714509204
Model Perplexity:  786.6139091762855


In [69]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')

In [66]:
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
0      150.222153  -72.269554       1        1  42.641849
3       51.163364 -159.968018       2        1  21.174904
1      -73.144554   21.523815       3        1  13.397385
2      -66.410133 -110.606438       4        1  12.891475
4       54.417324   12.443444       5        1   9.894386, topic_info=         Term          Freq         Total Category  logprob  loglift
1649  hijaako  15224.000000  15224.000000  Default  30.0000  30.0000
4660  twitter   1594.000000   1594.000000  Default  29.0000  29.0000
135       ang   2208.000000   2208.000000  Default  28.0000  28.0000
3304      pic   1534.000000   1534.000000  Default  27.0000  27.0000
688       com   1523.000000   1523.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
1346   friend    124.046892    209.467742   Topic5  -4.9621   1.7893
1649  hijaako    951.324958  15224.835942   Topic5  -2.9249  -0.4596
4622   tshirt     97.903992    143.504903   Topic5  -5.1988   1.9308
3592   rapist    163.019759    880.619835   Topic5  -4.6889   0.6264
2636     mean     85.260503    146.413101   Topic5  -5.3370   1.7725

[265 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
100       2  0.999586    ako
101       2  0.994785  akong
105       2  0.994832   alam
135       2  0.321438    ang
135       4  0.678642    ang
...     ...       ...    ...
4989      1  0.996797  youre
4994      4  0.989240    yrs
4996      2  0.995445    yun
4997      2  0.963388   yung
4997      4  0.035824   yung

[312 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 2, 3, 5])

In [ ]:
# Get Log Likelyhoods from Grid Search Output
n_topics = [10, 15, 20, 25, 30]
log_likelyhoods_5 = [round(gscore.mean_validation_score) for gscore in model.grid_scores_ if gscore.parameters['learning_decay']==0.5]
log_likelyhoods_7 = [round(gscore.mean_validation_score) for gscore in model.grid_scores_ if gscore.parameters['learning_decay']==0.7]
log_likelyhoods_9 = [round(gscore.mean_validation_score) for gscore in model.grid_scores_ if gscore.parameters['learning_decay']==0.9]

# Show graph
plt.figure(figsize=(12, 8))
plt.plot(n_topics, log_likelyhoods_5, label='0.5')
plt.plot(n_topics, log_likelyhoods_7, label='0.7')
plt.plot(n_topics, log_likelyhoods_9, label='0.9')
plt.title("Choosing Optimal LDA Model")
plt.xlabel("Num Topics")
plt.ylabel("Log Likelyhood Scores")
plt.legend(title='Learning decay', loc='best')
plt.show()

In [ ]:
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_topics)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

In [ ]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

In [ ]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel